<a href="https://colab.research.google.com/github/gabiacuna/KL2021/blob/main/Fundamentos%202/3_logreg_nb_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NLP (Procesamiento del lenguaje)

Etiquetado de partes del discutrso y clasificando sentimientos.

### Google natural lenguage api

[link](https://cloud.google.com/natural-language?hl=es)

cursito introductorio ai : https://www.fast.ai/2019/07/08/fastai-nlp/

### Analisis de Sentimientos

Como expresamos nuestros sentimientos a travez del lenguaje. 

Pueden ser positivos, negativos y neutros.

Basado en el analisis de sentimientos, se crean compañias que revisan el comportamiento y recepciones de marcas en redes sociales. [ej](https://netbasequid.com/)

[Estudio de Tweets en elecciones presidenciales](https://repositorio.unal.edu.co/handle/unal/56482)

-> [crawling twitter medium](https://medium.com/analytics-vidhya/crawling-twitter-data-without-authentication-8269d1c5b261)



# Clasificación de Sentimientos de Críticas de Películas (utilizando Naive Bayes, Logistic Regression y n-gramas)

Este notebook es una adaptación del existente en el curso de Fast.ai de procesamiento del lenguaje natural https://github.com/fastai/course-nlp/blob/master/3-logreg-nb-imdb.ipynb

Se mostratán las siguientes técnicas de clasificación de sentimientos:

- Naive Bayes
- Logistic Regression
- ngrams

Se emplearán las siguientes librerías (no requieren instalación en colab): 

- *fastai* [the fastai library](https://docs.fast.ai) : Para instalarla utilizar `pip install -U scikit-learn`
- *sklearn* [the scikit-learn library](https://scikit-learn.org/stable/user_guide.html):  Para instalarla se aconseja utilizar: `conda install -c pytorch -c fastai fastai=1.0` ó 
`pip install fastai==1.0`


## IMDB dataset

Este dataset ([large movie review dataset](http://ai.stanford.edu/~amaas/data/sentiment/))  contiene una colección de 50.000 críticas de IMDB y Fast.ai aloja estos dataset en AWS [fast.ai datasets](https://course.fast.ai/datasets.html). 

En la versión de fastai, consideraron críticas que tienen una alta polarización:

- Una crítica negativa, es aquel que contiene un score ≤ 4/10
- Una crítica positiva es aquella que tiene un puntaje ≥ 7/10
- No se incluyeron críticas neutrales.


Usualmente el dataset - y en este caso se aplica -, está dividido en dos colecciones. 
- Una de entrenamiento *training* 
- Otra de pruebas *testing*

La tarea de **clasificación de sentimientos** consiste en predecir la polaridad (que tan positivo o que tan negativo es un texto dado).


### Imports

En esta parte se importarán las librerías a emplear en este notebook:

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from fastai import *
from fastai.text import *

In [6]:
import sklearn.feature_extraction.text as sklearn_text

### Tokenización y creación de la matriz de Documento-término

fast.ai tiene una colección de datasets [datasets hosted via AWS Open Datasets](https://course.fast.ai/datasets.html) para descarga rápida usando URLs. 


In [7]:
#?? URLs

Es una buena idea empezar con un dataset de ejemplo antes de utilizar el dataset completo para realizar cómputos rápidos y hacer que el modelo funcione. Así se puede obtener un dataset de ejemplo:

In [8]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

Es posible apreciar el contenido del dataframe usando pandas:

In [9]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


En este notebook se utilizará [TextList](https://docs.fast.ai/text.data.html#TextList) que se incluye en la librería fastai:

In [11]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                         .split_from_df(col=2)
                         .label_from_df(cols=0))

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


### Exploración de los datos

Un punto de inicio para cualquier problema que involucre datos es explorarlos y ver como lucen. En este caso vamos a análizar críticas de películas que han sido **etiquetadas** como **positivas** o **negativas.** 


In [14]:
movie_reviews.valid.x[0], movie_reviews.valid.y[0] 

(Text [ 2  5 21 71 ... 15  5  0 52], Category 1)

Se genera el mapeo de token a entero

En NLP, un **token** es la unidad básica de procesamiento (el concepto de token depende de la aplicación y de lo que se quiera realizar). En esta parte, un token hace referencia a palabras o a signos de puntuación y también tendremos algunos tokens especiales que corresponden a palabras desconocidas, mayúsculas, etc.  Los tokens que empiezan con "xx" son especiales y son generados por fast.ai. En la ([documentación de fast.ai](https://fastai1.fast.ai/text.transform.html#Tokenizer)) se puede apreciar una lista de tokens y sus significados: 

![](https://raw.githubusercontent.com/arleserp/CCE2021/main/images/tokens.png)


Podemos apreciar que el resultado del procesamiento en fast.ai de el dataset de ejemplo genera dos dataset uno de entrenamiento con 800 registros y otro de validación con 200:

In [16]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

Adicionalmente se generan una lista y un diccionario. La lista incluye un mapeo de enteros a tokens `(movie_reviews.vocab.itos)` y el diccionario mapea de tokens a enteros (`movie_reviews.vocab.stoi`). ¿Por qué cree que son de diferentes tamaños?: 

In [17]:
print(type(movie_reviews.vocab.itos), type(movie_reviews.vocab.stoi))
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

<class 'list'> <class 'collections.defaultdict'>


(6008, 19159)

Se mapean las palabras a token (a cada palabra se le asigna un numero). Si llega una palabra nueva se usa el mismo token.

En int to string se guarda {token : palabra}

Y en string to int se guarda {palabra : token}

Si miramos que número de token tiene la palabra language:

In [28]:
movie_reviews.vocab.stoi['language'] 

917

Si miramos la palabra a la que corresponde el token 917:

In [29]:
movie_reviews.vocab.itos[917]

'language'

Las palabras de la 20 a la 29 son:

In [30]:
movie_reviews.vocab.itos[20:30]

['that', 'this', '"', "'s", '\n \n ', '-', 'was', 'as', 'for', 'movie']

La palabra 6007 es:

In [31]:
movie_reviews.vocab.itos[6007]

'sollett'

In [32]:
movie_reviews.vocab.itos[:20] #primeras 20 palabras

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i']

In [33]:
movie_reviews.vocab.stoi

defaultdict(int,
            {'xxunk': 0,
             'xxpad': 1,
             'xxbos': 2,
             'xxeos': 3,
             'xxfld': 4,
             'xxmaj': 5,
             'xxup': 6,
             'xxrep': 7,
             'xxwrep': 8,
             'the': 9,
             '.': 10,
             ',': 11,
             'and': 12,
             'a': 13,
             'of': 14,
             'to': 15,
             'is': 16,
             'it': 17,
             'in': 18,
             'i': 19,
             'that': 20,
             'this': 21,
             '"': 22,
             "'s": 23,
             '\n \n ': 24,
             '-': 25,
             'was': 26,
             'as': 27,
             'for': 28,
             'movie': 29,
             'with': 30,
             'but': 31,
             'film': 32,
             'you': 33,
             ')': 34,
             'on': 35,
             '(': 36,
             "n't": 37,
             'are': 38,
             'he': 39,
             'his': 40,
       

Let's test that a non-word maps to xxunk:

In [34]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['rrachell']]

'xxunk'

In [35]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['language']]

'language'

In [36]:
t = movie_reviews.train[0][0]

In [37]:
t.data[:30]

array([   2,    5, 4619,   25,    0,   25,  867,   52,    5, 3776,    5, 1800,   95,   37,   85,  192,   64,  935,
          0, 2738,  517,   18,   21,   11,   84, 2417,  193,   88, 3777,   64])

## Creación manual de nuestra matriz de término-documento

Esta matriz representa una bolsa de palabras. No se mantiene un registro de como las palabras están ordenadas, solamente cuales palabras ocurren y que tan a menudo.

Es posible utilizar [sklearn's CountVectorizer](https://github.com/scikit-learn/scikit-learn/blob/55bf5d9/sklearn/feature_extraction/text.py#L940). En este notebook se creará de forma manual para:
- entender lo que sklearn está haciendo por debajo
- crear algo que se integre con TextList de fastai


Para crear esta matriz es necesario entender un poco como funcionan los **Counter** de python y las matrices **dispersas**.


### Counters

Un counter permite contar la cantidad de ocurrencias de una lista.

In [42]:
c = Counter([4,2,8,8,4,8]) 

In [43]:
c

Counter({2: 1, 4: 2, 8: 3})

In [44]:
c.values()

dict_values([2, 1, 3])

In [45]:
c.keys()

dict_keys([4, 2, 8])

Counter pertenece al modulo de collections de python (como OrderedDict, defaultdict, deque, and namedtuple).

### Matrices disperas en Scipy (in Scipy)

Aunque se han reducido de 19.000 palabras a 6000. La mayoría de los token no aparecen en la mayoría de críticas. Esto nos permite tomar ventaja y almacenar los datos en una **matriz dispersa**.

- Una matriz con muchos ceros es dispersa (**sparse**)
- Lo opuesto a una matriz dispersa es una matriz densa (**dense**)
- Para optimizar memoria es posible guardar únicamente los datos que no son cero

Para ahorrar memoria se utilizan las matrices dispersas.


<img src="https://dziganto.github.io/assets/images/sparse_matrix.png?raw=true" alt="floating point" style="width: 30%"/>

Otro ejemplo de una matriz dispersa grande:

<img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Finite_element_sparse_matrix.png" tyle="width: 20%"/>

[Source](https://commons.wikimedia.org/w/index.php?curid=2245335)

Los formatos más comunes de almacenamiento de matrices dispersas son:
- coordinate-wise (scipy calls COO)
- compressed sparse row (CSR) *
- compressed sparse column (CSC)

-> fornmatos en los que se almacena la matriz para ahorrar memoria

[Aquí algumnos ejemplos](http://www.mathcs.emory.edu/~cheung/Courses/561/Syllabus/3-C/sparse.html)

[Aquí muchos otros formatos](http://www.cs.colostate.edu/~mcrob/toolbox/c++/sparseMatrix/sparse_matrix_compression.html)

### Nuestro CountVectorizer

In [46]:
Counter((movie_reviews.valid.x)[0].data)

Counter({0: 32,
         2: 1,
         5: 32,
         6: 1,
         9: 10,
         10: 7,
         11: 10,
         12: 1,
         13: 4,
         14: 6,
         15: 6,
         16: 4,
         18: 2,
         20: 1,
         21: 3,
         23: 1,
         24: 3,
         25: 2,
         26: 1,
         27: 3,
         30: 1,
         44: 1,
         45: 1,
         49: 1,
         50: 3,
         52: 1,
         54: 2,
         58: 1,
         59: 1,
         63: 2,
         71: 1,
         74: 1,
         77: 1,
         84: 1,
         109: 1,
         115: 1,
         149: 1,
         190: 1,
         194: 1,
         197: 2,
         204: 1,
         207: 1,
         221: 1,
         239: 1,
         251: 1,
         258: 1,
         285: 1,
         288: 1,
         319: 1,
         324: 1,
         337: 1,
         358: 1,
         378: 1,
         404: 1,
         409: 1,
         430: 1,
         456: 1,
         478: 1,
         541: 1,
         571: 1,
         579: 1

In [52]:
movie_reviews.vocab.itos[71]

'very'

In [53]:
(movie_reviews.valid.x)[0]

Text [ 2  5 21 71 ... 15  5  0 52]

In [54]:
def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)

    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
        
#     return (values, j_indices, indptr)

    return scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, vocab_len),
                                   dtype=int)

In [55]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

CPU times: user 57.3 ms, sys: 551 µs, total: 57.9 ms
Wall time: 59.2 ms


In [56]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

CPU times: user 179 ms, sys: 3.44 ms, total: 183 ms
Wall time: 184 ms


In [57]:
trn_term_doc.shape

(800, 6008)

In [58]:
trn_term_doc[:,-10:]

<800x10 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [59]:
val_term_doc.shape

(200, 6008)

### Exploración adicional de datos:

Podemos convertir nuestra matriz de una representación dispersa a una representación fila columna.

In [60]:
movie_reviews.vocab.itos[-1:] #ultima palabra de la lista itos

['sollett']

In [66]:
val_term_doc.todense()[:10,:10]

matrix([[32,  0,  1,  0, ...,  1,  0,  0, 10],
        [ 9,  0,  1,  0, ...,  1,  0,  0,  7],
        [ 6,  0,  1,  0, ...,  0,  0,  0, 12],
        [78,  0,  1,  0, ...,  0,  0,  0, 44],
        ...,
        [ 8,  0,  1,  0, ...,  0,  0,  0,  8],
        [43,  0,  1,  0, ...,  8,  1,  0, 25],
        [ 7,  0,  1,  0, ...,  1,  0,  0,  9],
        [19,  0,  1,  0, ...,  2,  0,  0,  5]])

In [64]:
movie_reviews.vocab.itos[-1]

'sollett'

In [67]:
review = movie_reviews.valid.x[1]
review

Text [  2  19 248  21 ...   9   0  10   0]

**Ejercicio:**

La palabra *late* aparece dos veces en esta crítica. Confirme que un valor de 2 es almanenado en la matriz de termino-documento.


#### Respuesta (Programe la respuesta aquí):

In [70]:
# Exercise: Confirm this

val_term_doc.todense()[1,movie_reviews.vocab.stoi['late']]

#Matriz [en la fila 1, columna de late]

2

In [ ]:
val_term_doc

<200x6008 sparse matrix of type '<class 'numpy.int32'>'
	with 27848 stored elements in Compressed Sparse Row format>

In [ ]:
val_term_doc[1]

<1x6008 sparse matrix of type '<class 'numpy.intc'>'
	with 81 stored elements in Compressed Sparse Row format>

In [ ]:
val_term_doc[1].sum()

144

La crítica tiene en total 81 tokens distintos con 144 en total.

In [71]:
review.data

array([  2,  19, 248,  21, ...,   9,   0,  10,   0])

**Ejercicio 2:** Como convertir review.data en texto sin usar review.text?

#### Respuesta

In [76]:
# Exercise

text = ''
for i in review.data :
    text += movie_reviews.vocab.itos[i] + ' '
print(text)

xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
 
  xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
 
  xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk 


**Ejercicio 3**: Confirme que review tiene exactamente 81 tokens

#### Answer

In [82]:
# Escriba aquí la respuesta

#sol mia
len(np.unique(review.data))

#sol profe
len(set(review.data))

81

## ¿Por qué stoi tiene más elementos que itos?

In [83]:
movie_reviews.vocab.itos[1000:1005]

['state', 'street', 'impossible', 'clever', 'development']

`stoi` (string-to-int) is larger than `itos` (int-to-string).

In [84]:
len(movie_reviews.vocab.stoi) - len(movie_reviews.vocab.itos)

13152

Hay muchas palabras que mapean unknown:

In [85]:
unk = []
for word, num in movie_reviews.vocab.stoi.items():
    if num == 0:
        unk.append(word)

In [86]:
len(unk)

13153

In [90]:
unk[:6]

['xxunk', 'bleeping', 'pert', 'ticky', 'schtick', 'whoosh']

## Naive Bayes

Definimos la relación logarítmica **log-count ratio** $r$ para cada palabra $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

Donde $f$ es el número de veces que un documento positivo/negativo tiene una característica dividida sobre el número de documentos positivos/negativos respectivamente.

In [91]:
movie_reviews.y.classes

['negative', 'positive']

In [92]:
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [93]:
positive = y.c2i['positive'] #obtiene clase a index para positivo 1
negative = y.c2i['negative'] #obtain clase a index para negativo 0

In [94]:
?? x[y.items==positive].sum(0)

Para cada palabra se puede calcular un p1 y p0, que son la cantidad de veces que se repite en las reviews positivas o negarivas, una palabra esta x veces en reviews positivos (p1) o negativos (p0). Con esto podemos identificar si una palabra es positiva o negativa.

In [97]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0))) 
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [96]:
len(p1), len(p0)

(6008, 6008)

In [98]:
np.squeeze(np.asarray(x[y.items==negative].sum(0))) #conteos de una palabra totales en críticas negativas

array([7153,    0,  417,    0, ...,    0,    3,    3,    3], dtype=int64)

In [99]:
np.asarray(x[y.items==positive].sum(0)) #retorna una lista de listas

array([[6471,    0,  383,    0, ...,    3,    0,    0,    0]], dtype=int64)

In [100]:
np.squeeze(np.asarray(x[y.items==positive].sum(0))) #conteos de una palabra totales en críticas positivas, squeeze elimina esa dimensión adicional

array([6471,    0,  383,    0, ...,    3,    0,    0,    0], dtype=int64)

Para cada palabra en nuestro vocabulario estamos sumando en cuantas críticas positivas y en cuantas negativas están.

In [ ]:
p1[:10]

array([ 6471,     0,   383,     0,     0, 10267,   674,    57,     0,  5260], dtype=int32)

In [101]:
v = movie_reviews.vocab

In [102]:
v.itos[0]

'xxunk'

In [103]:
v.itos[6004]

'coaxes'

### Usando los ratios para explorar datos

Se puede utilizar p0 y p1 para saber cuantas veces aparece una palabra dada en críticas positivas vs cuantas veces aparece en críticas negativas.

**Ejercicio**: Que tan a menudo aparece **loved** en críticas positivas vs. en críticas negativas... y **hated**?

#### Answer:

In [114]:
# Escriba aquí la relación para loved.
indexL = movie_reviews.vocab.stoi['loved']
indexH = movie_reviews.vocab.stoi['hated']

print('Loved\t p0', p0[indexL], 'p1', p1[indexL])

Loved	 p0 12 p1 29


In [113]:
# Escriba aquí la relación para hated.
print('Hated\t p0', p0[indexH], 'p1', p1[indexH])

Hated	 p0 6 p1 3


#### Obtener reviews positivos para la palabra hated

Es posible hacerlo:

In [115]:
v.stoi['hated']

1977

In [116]:
a = np.argwhere((x[:,1977] > 0))[:,0]
a

array([ 15,  49, 304, 351, 393, 612, 695, 773], dtype=int32)

In [117]:
b = np.argwhere(y.items==positive)[:,0]
b

array([  1,   3,  10,  11, ..., 787, 789, 790, 797])

In [118]:
set(a).intersection(set(b))

{393, 612, 695}

In [119]:
review = movie_reviews.train.x[695]
review.text

"xxbos xxmaj xxunk , yeah this episode is extremely underrated . \n \n  xxmaj even though there is a xxup lot of bad writing and acting at parts . i think the good over wins the bad . \n \n  i love the xxunk parts and the big ' twist ' at the end . i absolutely love that scene when xxmaj michelle xxunk xxmaj tony . xxmaj it 's actually one of my favorite scenes of xxmaj season 1 . \n \n  xxmaj for some reason , people have always hated the xxmaj xxunk episodes , yet i have always liked them . xxmaj they 're not the best , in terms of writing . but the theme really does interest me , \n \n  i 'm gon na give it a xxup three star , but if the writing were a little more consistent i 'd give it xxup four ."

#### Obtener críticas negativas con la palabra "loved"

Miremos algunas críticas negativas con la palabra loved

In [120]:
v.stoi['loved']

535

In [121]:
a = np.argwhere((x[:,535] > 0))[:,0]    #Me traigo todos los reviews con la palabra loved
a

array([  1,  15,  29,  69,  75,  79, 174, 185, 200, 205, 262, 296, 303, 333, 350, 351, 398, 407, 440, 489, 496, 528,
       538, 600, 602, 605, 627, 642, 657, 660, 700, 712, 729, 735, 755, 767, 785], dtype=int32)

In [126]:
b = np.argwhere(y.items==negative)[:,0] #Me traigo todos los reviews negativos
b

array([  0,   2,   4,   5, ..., 795, 796, 798, 799])

In [123]:
set(a).intersection(set(b)) #Reviews negativos con la palabra loved

{15, 200, 205, 303, 351, 398, 600, 605, 642, 700, 729, 767}

In [125]:
review = movie_reviews.train.x[200]
review.text

'xxbos i loved the first " xxmaj azumi " movie . i \'ve seen xxmaj ms. xxmaj ueto in a variety of her xxup tv appearances and i \'ve seen my fair share of samurai and ninja flicks . i have to say that this movie was much weaker than i \'d expected . \n \n  xxmaj given the movie \'s cast and set up in " xxmaj azumi " , they should have been able to do a much better job with this movie , but instead it was slow , xxunk in parts , and xxunk with very poor , unconvincing , and wooden acting . \n \n  xxmaj when they bothered to reference the first movie , they did so in a manner that was pretty loose and weak . xxmaj in " xxmaj azumi " , the title character is the best of a group of superior killers . xxmaj in " xxmaj azumi 2 " she seems somehow xxunk and less - impressive . \n \n  xxmaj that \'s not to say it was a total loss . xxmaj there were a few decent fight scenes and some over - the - top characters . xxmaj unfortunately , the movie suffers overall from the simple fact that xxmaj xx

## Aplicando Naive Bayes

In [127]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

Divido los p0 y p1 por la cantidad de reviews negativas y positivas  +1 .

In [128]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

r nos dice que tan positiva o negativa es una palabra $r \in [-1, 1]$.

In [130]:
r = np.log(pr1/pr0); r 

array([-0.015348,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

### Vocabulario asociado a críticas positivas o negativas

In [131]:
biggest = np.argpartition(r, -10)[-10:] #obtiene los 10 valores más grandes (críticas más positivas)
smallest = np.argpartition(r, 10)[:10]  #obtiene los valores más pequeños (críticas más negativas)

Palabras más positivas:

In [132]:
[v.itos[k] for k in biggest]

['han',
 'jabba',
 'gilliam',
 'davies',
 'noir',
 'felix',
 'jimmy',
 'astaire',
 'fanfan',
 'biko']

In [133]:
np.argmax(trn_term_doc[:,v.stoi['biko']])

515

In [139]:
movie_reviews.train.x[515].text

'xxbos " xxmaj the xxmaj true xxmaj story xxmaj of xxmaj the xxmaj friendship xxmaj that xxmaj shook xxmaj south xxmaj africa xxmaj and xxmaj xxunk xxmaj the xxmaj world . " \n \n  xxmaj richard xxmaj attenborough , who directed " a xxmaj bridge xxmaj too xxmaj far " and " xxmaj gandhi " , wanted to bring the story of xxmaj steve xxmaj biko to life , and the journey and trouble that xxunk xxmaj donald xxmaj woods went through in order to get his story told . xxmaj the films uses xxmaj wood \'s two books for it \'s information and basis - " xxmaj biko " and " xxmaj asking for xxmaj trouble " . \n \n  xxmaj the film takes place in the late 1970 \'s , in xxmaj south xxmaj africa . xxmaj south xxmaj africa is in the grip of the terrible apartheid , which keeps the blacks separated from the whites and xxunk the whites as the superior race . xxmaj the blacks are forced to live in xxunk on the xxunk of the cities and xxunk , and they come under frequent xxunk by the police and the army . xxma

Most negative words:

In [ ]:
[v.itos[k] for k in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

In [142]:
np.argmax(trn_term_doc[:,v.stoi['dog']])

547

In [143]:
movie_reviews.train.x[547].text

"xxbos a dog found in a local kennel is xxunk with xxmaj satan and has a xxunk of xxunk , one of which is given to a family who has just lost their previous dog to a hit & run . xxmaj the puppy wants no time in making like xxmaj donald xxmaj xxunk and firing the xxmaj mexican xxunk , how xxunk . xxmaj only the father suspects that this xxunk is more then he appears , the rest of the family loves the demonic xxunk . xxmaj so it 's up to dad to say the day . \n \n  xxmaj this late 70 's made for xxup tv horror flick has little going for it except a misplaced feeling of nostalgia . xxmaj when i saw this as a kid i found it to be a tense nail - xxunk , but xxunk it as an adult i now realize that it 's merely lame , boring , and not really well - acted in the least bit . \n \n  xxmaj my xxmaj grade : d"

In [144]:
trn_term_doc[:,v.stoi['dog']]

<800x1 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [145]:
[v.itos[k] for k in smallest]

['vargas',
 'disappointment',
 'crater',
 'porn',
 'crap',
 'worst',
 'dog',
 'naschy',
 'soderbergh',
 'fuqua']

### Naive Bayes: continuación

In [153]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [154]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()) #normaliza para encontrar la razón logarítmica del número promedio de críticas 

In [157]:
preds = (val_term_doc @ r + b) > 0  #Multi los valores por el r + b de normalización, > 0 == true
preds   #Indica si cada evaluacion es positiva o negativa

array([ True, False, False, False, ...,  True, False, False,  True])

In [152]:
(preds == val_y.items).mean() #promedio-> de los que quedaron bien clasificados sobre el total

0.645

## Pasando al dataset completo 

Dado que funcionó con el data set de ejemplo pasamos a probar el dataset completo:

### Download data and process

In [158]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/root/.fastai/data/imdb/README'),
 PosixPath('/root/.fastai/data/imdb/unsup'),
 PosixPath('/root/.fastai/data/imdb/tmp_lm'),
 PosixPath('/root/.fastai/data/imdb/train'),
 PosixPath('/root/.fastai/data/imdb/tmp_clas'),
 PosixPath('/root/.fastai/data/imdb/test'),
 PosixPath('/root/.fastai/data/imdb/imdb.vocab')]

In [159]:
(path/'train').ls()

[PosixPath('/root/.fastai/data/imdb/train/unsupBow.feat'),
 PosixPath('/root/.fastai/data/imdb/train/pos'),
 PosixPath('/root/.fastai/data/imdb/train/labeledBow.feat'),
 PosixPath('/root/.fastai/data/imdb/train/neg')]

In [160]:
reviews_full = (TextList.from_folder(path)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos']))
             #label them all with their folders

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [161]:
len(reviews_full.train), len(reviews_full.valid)

(25000, 25000)

We will store the vocab in a variable `v` since we will be using it frequently:

In [162]:
v = reviews_full.vocab

In [163]:
v.itos[100:110]

['people',
 'bad',
 'will',
 'other',
 'also',
 'into',
 'first',
 'great',
 'because',
 'how']

In [164]:
%%time
val_term_doc = get_term_doc_matrix(reviews_full.valid.x, len(reviews_full.vocab.itos))

CPU times: user 6.03 s, sys: 365 ms, total: 6.4 s
Wall time: 5.94 s


In [165]:
%%time
trn_term_doc = get_term_doc_matrix(reviews_full.train.x, len(reviews_full.vocab.itos))

CPU times: user 6.13 s, sys: 328 ms, total: 6.46 s
Wall time: 6 s


### Guardar y cargar la matriz de término-documento

Como el proceso es lento es una muy buena idea guardar la matriz y después cargarla.

In [166]:
scipy.sparse.save_npz("trn_term_doc.npz", trn_term_doc)

In [167]:
scipy.sparse.save_npz("val_term_doc.npz", val_term_doc)

Cuando se guarde debe añadirse a git.ignore en caso de manejar git.

Así se cargan los datos:

In [168]:
trn_term_doc = scipy.sparse.load_npz("trn_term_doc.npz")
val_term_doc = scipy.sparse.load_npz("val_term_doc.npz")

### Naive Bayes on full dataset

In [169]:
x=trn_term_doc
y=reviews_full.train.y

val_y = reviews_full.valid.y.items

In [170]:
x

<25000x38440 sparse matrix of type '<class 'numpy.int64'>'
	with 3716349 stored elements in Compressed Sparse Row format>

In [171]:
positive = y.c2i['pos']
negative = y.c2i['neg']

In [172]:
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [173]:
p1[:20]

array([ 28357,      0,  12500,      0,      0, 342615,  20462,   1338,      7, 173123, 138129, 143772,  89570,  83404,
        76828,  66715,  58510,  47901,  50177,  40455], dtype=int64)

### Exploración de datos: razones de una palabra en críticas positivas y negativas

Es posible obtener la razón entre el número de veces una palabra dada aparece en críticas negativas en positivas. Razones ( > 1) significan que la palabra indica un review negativo y valores (< 1) indican un review positivo.

In [178]:
def neg_pos_given_word(word):
    print(p0[v.stoi[word]]/p1[v.stoi[word]])

In [179]:
neg_pos_given_word('hated')

2.051546391752577


In [180]:
neg_pos_given_word('liked')

0.6424702058504875


In [181]:
neg_pos_given_word('loved')

0.3139963167587477


In [182]:
neg_pos_given_word('best')

0.48527706932529563


In [183]:
neg_pos_given_word('worst')

9.837301587301587


In [184]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [185]:
r = np.log(pr1/pr0)

In [186]:
r[v.stoi['hated']]

-0.7133498878774648

In [187]:
r[v.stoi['loved']]

1.1563661500586044

In [188]:
r[v.stoi['worst']]

-2.2826243504315076

In [189]:
r[v.stoi['best']]

0.7227894355186196

### Naive Bayes sobre el data set

In [174]:
negative = y.c2i['neg']
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

Como tenemos un número igual de críticas positivas y negativas b es 0.

In [175]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [176]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()); b

0.0

In [191]:
preds = (val_term_doc @ r + b) > 0

Nuestro accuracy es de 80% para el dataset completo:

In [192]:
(preds == val_y).mean()

0.80864

### Naive Bayes Binarizado

Maybe it only matters whether a word is in the review or not (not the frequency of the word):

In [193]:
x=trn_term_doc.sign()
y=reviews_full.train.y

In [194]:
x.todense()[:10,:10]

matrix([[1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 1, 0, 1],
        ...,
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [0, 0, 1, 0, ..., 0, 0, 0, 1]])

In [195]:
negative = y.c2i['neg']
positive = y.c2i['pos']

In [196]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [197]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [198]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

preds = (val_term_doc.sign() @ r + b) > 0

In [199]:
(preds==val_y).mean()

0.82932

## Regresión logística

Es posible ejecutar un modelo de regresión logística cuando las características son unigramas.

In [200]:
from sklearn.linear_model import LogisticRegression

In [201]:
m = LogisticRegression(C=0.1, dual=True, solver='liblinear')
m.fit(x, y.items.astype(int))   #Matriz de termino y clases
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.76988

Le gano el naive base a la regrecion logistica con frecuencia

Y la versión binaria:

In [202]:
m = LogisticRegression(C=0.1, dual=True, solver="liblinear")
m.fit(trn_term_doc.sign(), y.items.astype(int))
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.88544

# Trigramas con Naive Bayes 

A continuación se muestra un modelo de regresión logística con Naive Bayes desarrollado [aquí](https://www.aclweb.org/anthology/P12-2018). Para cada documento computamos atributos binarizados como se describió en la parte superior pero esta vez utilizando bigramas y trigramas. Cada característica es un log-count ratio. Un modelo de regresión logística es entrenado para predecir sentimientos.

### n-gramas

Un n-grama es una secuencia continua de ítmes (donde un item es una letra, una sílaba o una palabra). Un 1-grama es un unigrama, un 2-gram es un bigrama y un 3-gram es un trigrama

En este caso son secuencias contínuas de palabras como: "the dog", "said that" "can't you".

Es el mismo proceso pero considera el orden de las palabras, n palabras. Considera todos los posibles casos de ordenes consecutivos.

In [203]:
path = untar_data(URLs.IMDB_SAMPLE)

In [204]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0))

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [205]:
v = movie_reviews.vocab.itos

In [206]:
vocab_len = len(v)

## Nuestros datos

### Creación de la matriz de entrenamiento

In [207]:
min_n=1
max_n=3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

Iteramos sobre todas las secuencias de palabras para crear los n-gramas:

In [208]:
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) not in ngramtoi:
                if len(ngram)==1:
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

Es útil emplear diccionarios para convertir entre índices y strings (en este caso n-gramas). Tenemos dos estructuras generadas del código anterios `itongram` (index to n-gram) y `ngramtoi` (n-gram to index).

In [209]:
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [210]:
train_ngram_doc_matrix

<800x260382 sparse matrix of type '<class 'numpy.int64'>'
	with 678912 stored elements in Compressed Sparse Row format>

### Revisando los datos

In [211]:
len(ngramtoi), len(itongram)    #Cant de trigramas que nos salieron

(260382, 260382)

In [212]:
itongram[20005]

array([ 15,   9, 710])

In [213]:
ngramtoi[str(np.array([15,   9,  710]))]

20005

In [214]:
itongram[100000]

array([  24, 2883])

In [215]:
v[15], v[9], v[710]

('to', 'the', 'leads')

In [216]:
itongram[100010]

array([1450,   52])

In [217]:
v[5430], v[10]

('photographer', '.')

In [218]:
itongram[6116]

array([ 85, 192,  64])

In [219]:
v[85], v[191], v[64]

('even', 'nothing', 'her')

In [220]:
itongram[80000]

array([ 67, 177,  96])

In [221]:
v[2594], v[14], v[2618]

('loss', 'of', 'sleep')

### Cración de matriz de validación

In [227]:
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [228]:
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [229]:
valid_ngram_doc_matrix

<200x260382 sparse matrix of type '<class 'numpy.int64'>'
	with 121600 stored elements in Compressed Sparse Row format>

In [230]:
train_ngram_doc_matrix

<800x260382 sparse matrix of type '<class 'numpy.int64'>'
	with 678912 stored elements in Compressed Sparse Row format>

### Guardar matriz de n-gramas


In [231]:
scipy.sparse.save_npz("train_ngram_matrix.npz", train_ngram_doc_matrix)

In [232]:
scipy.sparse.save_npz("valid_ngram_matrix.npz", valid_ngram_doc_matrix)

In [233]:
with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ngramtoi.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Cargar matriz de n-gramas

In [234]:
train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_matrix.npz")

In [235]:
with open('itongram.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
with open('ngramtoi.pickle', 'rb') as handle:
    b = pickle.load(handle)

## Naive Bayes

In [236]:
x=train_ngram_doc_matrix
y=movie_reviews.train.y

In [237]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [238]:
x

<800x260382 sparse matrix of type '<class 'numpy.int64'>'
	with 678912 stored elements in Compressed Sparse Row format>

In [239]:
k=260373

In [240]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]

In [241]:
xx = x[:k]

In [242]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [243]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [244]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [245]:
r = np.log(pr1/pr0)

In [246]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

In [247]:
b

-0.08505123261815539

In [248]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [249]:
pre_preds = valid_ngram_doc_matrix @ r.T + b

In [250]:
pre_preds

array([ 111.020095,   39.757212,    1.56591 ,   14.717061, ...,   81.738199,   -5.930168, -152.211735,  120.253329])

In [251]:
preds = pre_preds.T>0

In [252]:
preds[:10]

array([ True,  True,  True,  True, False,  True,  True, False,  True, False])

In [253]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [254]:
(preds == valid_labels).mean()

0.76

### Naive Bayes Binarizado

In [255]:
trn_x_ngram_sgn = train_ngram_doc_matrix.sign()
val_x_ngram_sgn = valid_ngram_doc_matrix.sign()

In [256]:
xx = trn_x_ngram_sgn[:k]

In [257]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [258]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [259]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

pre_preds = val_x_ngram_sgn @ r.T + b
preds = pre_preds.T>0

In [260]:
(preds==valid_labels).mean()

0.735

## Regresión Logística

Aquí ajustamos la regresión logística regularizada donde las características son los trigramas.

In [261]:
from sklearn.linear_model import LogisticRegression

### usando CountVectorizer 

In [262]:
from sklearn.feature_extraction.text import CountVectorizer

In [263]:
veczr = CountVectorizer(ngram_range=(1,3), preprocessor=noop, tokenizer=noop, max_features=800000)

In [264]:
docs = movie_reviews.train.x

In [265]:
train_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.train.x]

In [266]:
valid_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.valid.x]

In [267]:
%%time
train_ngram_doc = veczr.fit_transform(train_words)

CPU times: user 1.79 s, sys: 8.32 ms, total: 1.8 s
Wall time: 1.8 s


In [268]:
train_ngram_doc

<800x260381 sparse matrix of type '<class 'numpy.int64'>'
	with 565703 stored elements in Compressed Sparse Row format>

In [269]:
veczr.vocabulary_

{'xxbos': 235222,
 'xxmaj': 235596,
 'un': 217518,
 '-': 14664,
 'xxunk': 247959,
 'believable': 50424,
 '!': 593,
 'meg': 134442,
 'ryan': 171960,
 'does': 72625,
 "n't": 141195,
 'even': 78285,
 'look': 129013,
 'her': 101678,
 'usual': 219408,
 'lovable': 129866,
 'self': 175871,
 'in': 110060,
 'this': 206626,
 ',': 8804,
 'which': 228205,
 'normally': 145186,
 'makes': 131550,
 'me': 133650,
 'forgive': 88666,
 'shallow': 177165,
 'acting': 27686,
 '.': 16840,
 'hard': 97859,
 'to': 210371,
 'believe': 50461,
 'she': 177313,
 'was': 222316,
 'the': 193805,
 'producer': 164488,
 'on': 152324,
 'dog': 72930,
 'plus': 162132,
 'kevin': 122608,
 'kline': 123465,
 ':': 20363,
 'what': 226862,
 'kind': 123235,
 'of': 147516,
 'suicide': 188360,
 'trip': 215815,
 'has': 98127,
 'his': 103530,
 'career': 58695,
 'been': 49238,
 '?': 20991,
 '...': 18346,
 'finally': 85372,
 'directed': 71216,
 'by': 56560,
 'guy': 96414,
 'who': 229030,
 'did': 70472,
 'big': 51599,
 'must': 140411,
 'be'

In [270]:
val_ngram_doc = veczr.transform(valid_words)

In [271]:
val_ngram_doc

<200x260381 sparse matrix of type '<class 'numpy.int64'>'
	with 93552 stored elements in Compressed Sparse Row format>

In [272]:
vocab = veczr.get_feature_names()

In [273]:
vocab[200000:200005]

['the same can',
 'the same car',
 'the same castle',
 'the same cat',
 'the same characters']

#### Naive Bayes binarizado usando ngrams y CountVectorizer

In [274]:
y=movie_reviews.train.y

C es la inversa de la fuerza de regularización; los valores más pequeños especifican una regularización más fuerte. Regularizado:

In [275]:
m = LogisticRegression(C=0.1, solver="liblinear", dual=True)
m.fit(train_ngram_doc.sign(), y.items);

preds = m.predict(val_ngram_doc.sign())
(preds.T==valid_labels).mean()

0.83

No binarizado:

In [276]:
m = LogisticRegression(C=0.1, solver="liblinear",dual=True)
m.fit(train_ngram_doc, y.items);

preds = m.predict(val_ngram_doc)
(preds.T==valid_labels).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.78

### Usando los ngrams, binarizado:

In [277]:
m2 = LogisticRegression(C=0.1, solver="liblinear", dual=True)
m2.fit(trn_x_ngram_sgn, y.items)

LogisticRegression(C=0.1, class_weight=None, dual=True, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [278]:
preds = m2.predict(val_x_ngram_sgn)
(preds.T==valid_labels).mean()

0.83

## References

* Baselines and Bigrams: Simple, Good Sentiment and Topic Classification. Sida Wang and Christopher D. Manning [pdf](https://www.aclweb.org/anthology/P12-2018)